In [2]:
#from pandas.io.data import DataReader
import pandas as pd
import numpy as np 
#import xgboost as xgb
import sklearn as sk
import sklearn.linear_model as lm
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation   import StratifiedKFold
import time
from datetime import datetime
import math
from math import log
from sklearn import metrics,preprocessing,cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from time import gmtime, strftime
import scipy
import sys
import sklearn.decomposition
from sklearn.metrics import mean_squared_error,f1_score,precision_score,recall_score,roc_auc_score,accuracy_score
from string import punctuation
from sklearn.neighbors import RadiusNeighborsRegressor, KNeighborsRegressor
import time
from scipy import sparse
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
import operator
from sklearn import svm
from ggplot import *
#
#from pyalgotrade import dataseries
#from pyalgotrade.technical import macd
#from pyalgotrade.technical import ma
#from pyalgotrade.technical import rsi
#from pyalgotrade.technical import cross

import talib
from talib.abstract import *
import pyfolio as pf
#from zipline.transforms.ta import EMA
import zipline as zl
from zipline.algorithm import TradingAlgorithm
import tushare as ts

In [6]:
%matplotlib inline

### 读取数据

In [60]:
#%%original data
N = 6
data = pd.read_csv('/Users/jianboxue/Documents/Research_Projects/Momentum/index_shanghai.csv',index_col = 'date',parse_dates = 'date')

#features owned by the day for predicting(include open)
data['month'] = data.index.month
data['week'] = data.index.week
data['weekofyear'] = data.index.weekofyear
data['day'] = data.index.day
data['dayofweek'] = data.index.dayofweek
data['dayofyear'] = data.index.dayofyear

donchian_channel_max = np.array([max(data['high'][max(i,20)-20:max(i,20)]) for i in range(len(data))])#the highest price in last n days
donchian_channel_min = np.array([min(data['low'][max(i,20)-20:max(i,20)]) for i in range(len(data))])
data['dcmaxod'] = (data['open']-donchian_channel_max)/donchian_channel_max
data['dcminod'] = (data['open']-donchian_channel_min)/donchian_channel_min

num_all = data.shape[1]


#features owned only by previous data(include close,high,low,vol)
data['price_change'] = (data['close']-data['open']) /data['open']
data['vol_change'] = 0
data['vol_change'][1:] = (data['vol'][1:].values-data['vol'][:-1].values) /data['vol'][:-1].values
data['ibs'] = (data['close']-data['low']) /(data['high']-data['low'])

data['dcmaxcd'] = (data['close']-donchian_channel_max)/donchian_channel_max
data['dcmincd'] = (data['close']-donchian_channel_min)/donchian_channel_min

#data['macd'] = MACD(data).macd
#data['macdsignal'] = MACD(data).macdsignal
#data['macdhist'] = MACD(data).macdhist

data['%R'] = (np.array([max(data['high'][max(i,14)-14:max(i,14)]) for i in range(len(data))])-data.close.values)/((np.array([max(data['high'][max(i,14)-14:max(i,14)]) for i in range(len(data))])-np.array([min(data['low'][max(i,14)-14:max(i,14)]) for i in range(len(data))])))#Williams %R is a momentum indicator The default setting for Williams %R is 14 periods, which can be days, weeks, months or an intraday timeframe.

/Applications/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Y is the target series to predict

In [100]:
y = [1 if data['close'][i]>data['open'][i] else 0 for i in range(len(data))]
y = y[N-1:]

n_windows = data.shape[0]-N+1
windows = range(n_windows)

In [63]:
#%% features of open,high,low,close,vol
d = np.array(data.ix[:,:5])
d = np.array([d[w:w+N].ravel() for w in windows])

#generated features for all days that can be used in training
d_na = np.array(data.ix[:,5:num_all])
d_na = np.array([d_na[w:w+N].ravel() for w in windows])

d_n = np.array(data.ix[:,num_all:])
d_n = np.array([d_n[w:w+N-1].ravel() for w in windows])

nday = 1500

d = d[len(data)- nday:]
d_na = d_na[len(data)- nday:]
d_n = d_n[len(data)- nday:]
y = np.array(y[len(data)- nday:])

In [101]:
#%%

def normalizeNday(stocks,N):
    def process_column(i):
        #Replaces all high/low/vol data with 0, and divides all stock data by the opening price on the first day
        if operator.mod(i, 5) == 1:
            return stocks[i] * 0
        if operator.mod(i, 5) == 2:
            return stocks[i] * 0
        if operator.mod(i, 5) == 4:
            return stocks[i] * 0
            #return np.log(stocks[:,i] + 1)
        else:
            return stocks[i] / stocks[0]
    #n = stocks.shape[0]
    stocks_dat =  np.array([ process_column(i) for i in range(N*5-4)]).transpose()
    #stocks_movingavgO9O10 = np.array([int(i > j) for i,j in zip(stocks_dat[:,45], stocks_dat[:,40])]).reshape((n, 1))
    #stocks_movingavgC9O10 = np.array([int(i > j) for i,j in zip(stocks_dat[:,45], stocks_dat[:,43])]).reshape((n, 1))
    #return np.hstack((stocks_dat, stocks_movingavgO9O10, stocks_movingavgC9O10))
    return stocks_dat
#%%
d_normalized = pd.DataFrame(np.hstack((np.array([normalizeNday(w,N) for w in d]),d_n,d_na)))

#remove constants
nunique = pd.Series([len(d_normalized[col].unique()) for col in d_normalized.columns], index = d_normalized.columns)
constants = nunique[nunique<2].index.tolist()    
for col in constants:
    del d_normalized[col]
d_normalized = np.array(d_normalized)

train = d_normalized[:int(len(d)*2/3.)]
train_y = y[:int(len(d)*2/3.)]
test = d_normalized[int(len(d)*2/3.):]
test_y = y[int(len(d)*2/3.):]

plt.scatter(d[:, (N-1)*5] / d[:, (N-1)*5-2],  d[:, (N-1)*5+3] / d[:, (N-1)*5])
plt.xlim((.8,1.2)); plt.ylim((.8,1.2))
plt.xlabel("Opening N / Closing N-1"); plt.ylabel("Closing N / Opening N-1")
plt.title("Correlation between interday and intraday stock movement")
plt.show()

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [96]:
d = np.array(data.ix[:,:5])
d = np.array([d[w:w+N].ravel() for w in windows])

In [99]:
_96

NameError: name '_96' is not defined

In [75]:
%magic